# Lab 1 - Creating the SQL Tables

In this lab, use `sqlalchemy` to create, populate, and query a table from the baseball database, as well as for the `super_hero_powers.csv` table.  

In [25]:
import pandas as pd
from dfply import mutate
from sqlalchemy import String, Integer, Boolean
from sqlalchemy import create_engine
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import sessionmaker
from sqlalchemy import select
from more_sqlalchemy import result_dicts
from sqlalchemy import func

## Part 1 - Baseball Managers

In this part of the lab, you will walk through the process of creating a manager table from [Lahman’s Baseball Database](http://www.seanlahman.com/baseball-archive/statistics/)

## Task 1 - Download, unzip, rename 

1. Download the baseball database linked above (save to desktop)
2. Unzip the file and rename to `baseball`
3. Load the `core/Managers.csv` file into a pandas `DataFrame` using `read_csv`
4. Inspect the `column` names and `dtypes`

In [2]:
managers = pd.read_csv("./databases/baseball/core/Managers.csv")
managers.head()

,playerID,yearID,teamID,lgID,inseason,G,W,L,rank,plyrMgr
0,wrighha01,1871,BS1,NaN,1,31,20,10,3.0,Y
1,woodji01,1871,CH1,NaN,1,28,19,9,2.0,Y
2,paborch01,1871,CL1,NaN,1,29,10,19,8.0,Y
3,lennobi01,1871,FW1,NaN,1,14,5,9,8.0,Y
4,deaneha01,1871,FW1,NaN,2,5,2,3,8.0,Y


In [3]:
managers.columns

Index(['playerID', 'yearID', 'teamID', 'lgID', 'inseason', 'G', 'W', 'L',
       'rank', 'plyrMgr'],
      dtype='object')

In [4]:
[(col, managers[col].is_unique) for col in managers]

[('playerID', False),
 ('yearID', False),
 ('teamID', False),
 ('lgID', False),
 ('inseason', False),
 ('G', False),
 ('W', False),
 ('L', False),
 ('rank', False),
 ('plyrMgr', False)]

In [5]:
managers = (managers >>
            mutate(id = managers.index))

In [6]:
managers.id.is_unique

True

In [7]:
managers.dtypes

playerID     object
yearID        int64
teamID       object
lgID         object
inseason      int64
G             int64
W             int64
L             int64
rank        float64
plyrMgr      object
id            int64
dtype: object

In [8]:
managers.shape

(3469, 11)

In [9]:
managers.head()

,playerID,yearID,teamID,lgID,inseason,G,W,L,rank,plyrMgr,id
0,wrighha01,1871,BS1,NaN,1,31,20,10,3.0,Y,0
1,woodji01,1871,CH1,NaN,1,28,19,9,2.0,Y,1
2,paborch01,1871,CL1,NaN,1,29,10,19,8.0,Y,2
3,lennobi01,1871,FW1,NaN,1,14,5,9,8.0,Y,3
4,deaneha01,1871,FW1,NaN,2,5,2,3,8.0,Y,4


#### Task 2 - Create a `sqlalchemy` types `dict`

In [10]:
sql_types = {'id':Integer,
             'playerID':String, 
             'plyrMgr':String,
             'teamID':String, 
             'lgID':String, 
             'yearID':Integer, 
             'inseason':Integer, 
             'G':Integer, 
             'W':Integer, 
             'L':Integer,
             'rank':Integer}

#### Task 4 - Create an `engine` and `schema`

# <font color='red'>Warning: Delete database before continuing</font>

In [12]:
mang_eng = create_engine("sqlite:///databases/baseball.db")
mang_eng.echo = True
schema = pd.io.sql.get_schema(managers, 'manager', keys='id', con=mang_eng, dtype=sql_types)
print(schema)


CREATE TABLE manager (
	"playerID" VARCHAR, 
	"yearID" INTEGER, 
	"teamID" VARCHAR, 
	"lgID" VARCHAR, 
	inseason INTEGER, 
	"G" INTEGER, 
	"W" INTEGER, 
	"L" INTEGER, 
	rank INTEGER, 
	"plyrMgr" VARCHAR, 
	id INTEGER NOT NULL, 
	CONSTRAINT manager_pk PRIMARY KEY (id)
)




In [13]:
mang_eng.execute(schema)

2019-01-28 08:26:44,589 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2019-01-28 08:26:44,591 INFO sqlalchemy.engine.base.Engine ()
2019-01-28 08:26:44,595 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2019-01-28 08:26:44,597 INFO sqlalchemy.engine.base.Engine ()
2019-01-28 08:26:44,600 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE manager (
	"playerID" VARCHAR, 
	"yearID" INTEGER, 
	"teamID" VARCHAR, 
	"lgID" VARCHAR, 
	inseason INTEGER, 
	"G" INTEGER, 
	"W" INTEGER, 
	"L" INTEGER, 
	rank INTEGER, 
	"plyrMgr" VARCHAR, 
	id INTEGER NOT NULL, 
	CONSTRAINT manager_pk PRIMARY KEY (id)
)


2019-01-28 08:26:44,604 INFO sqlalchemy.engine.base.Engine ()
2019-01-28 08:26:44,624 INFO sqlalchemy.engine.base.Engine COMMIT


#### Task 5 - Use `to_sql` with `if_exists='append'` to insert the data

In [14]:
managers.to_sql('manager', 
                con=mang_eng, 
                dtype=sql_types, 
                index=False,
                if_exists='append')

2019-01-28 08:26:48,660 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("manager")
2019-01-28 08:26:48,662 INFO sqlalchemy.engine.base.Engine ()
2019-01-28 08:26:48,667 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2019-01-28 08:26:48,758 INFO sqlalchemy.engine.base.Engine INSERT INTO manager ("playerID", "yearID", "teamID", "lgID", inseason, "G", "W", "L", rank, "plyrMgr", id) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
2019-01-28 08:26:48,759 INFO sqlalchemy.engine.base.Engine (('wrighha01', 1871, 'BS1', None, 1, 31, 20, 10, 3.0, 'Y', 0), ('woodji01', 1871, 'CH1', None, 1, 28, 19, 9, 2.0, 'Y', 1), ('paborch01', 1871, 'CL1', None, 1, 29, 10, 19, 8.0, 'Y', 2), ('lennobi01', 1871, 'FW1', None, 1, 14, 5, 9, 8.0, 'Y', 3), ('deaneha01', 1871, 'FW1', None, 2, 5, 2, 3, 8.0, 'Y', 4), ('fergubo01', 1871, 'NY2', None, 1, 33, 16, 17, 5.0, 'Y', 5), ('mcbridi01', 1871, 'PH1', None, 1, 28, 21, 7, 1.0, 'Y', 6), ('hastisc01', 1871, 'RC1', None, 1, 25, 4, 21, 9.0, 'Y', 7)  ... displaying 10 of

#### Task 6 - Query the table to make sure it all worked

In [15]:
mang_eng2 = create_engine("sqlite:///databases/baseball.db") 
Session = sessionmaker(mang_eng)
session = Session()

In [16]:
Base = automap_base()
Base.prepare(mang_eng2, reflect=True)
Manager = Base.classes.manager

In [17]:
stmt = select('*').select_from(Manager)
session.execute(stmt).fetchmany(5) >> result_dicts

2019-01-28 08:26:53,079 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2019-01-28 08:26:53,081 INFO sqlalchemy.engine.base.Engine SELECT * 
FROM manager
2019-01-28 08:26:53,084 INFO sqlalchemy.engine.base.Engine ()


[{'G': 31,
  'L': 10,
  'W': 20,
  'id': 0,
  'inseason': 1,
  'lgID': None,
  'playerID': 'wrighha01',
  'plyrMgr': 'Y',
  'rank': 3,
  'teamID': 'BS1',
  'yearID': 1871},
 {'G': 28,
  'L': 9,
  'W': 19,
  'id': 1,
  'inseason': 1,
  'lgID': None,
  'playerID': 'woodji01',
  'plyrMgr': 'Y',
  'rank': 2,
  'teamID': 'CH1',
  'yearID': 1871},
 {'G': 29,
  'L': 19,
  'W': 10,
  'id': 2,
  'inseason': 1,
  'lgID': None,
  'playerID': 'paborch01',
  'plyrMgr': 'Y',
  'rank': 8,
  'teamID': 'CL1',
  'yearID': 1871},
 {'G': 14,
  'L': 9,
  'W': 5,
  'id': 3,
  'inseason': 1,
  'lgID': None,
  'playerID': 'lennobi01',
  'plyrMgr': 'Y',
  'rank': 8,
  'teamID': 'FW1',
  'yearID': 1871},
 {'G': 5,
  'L': 3,
  'W': 2,
  'id': 4,
  'inseason': 2,
  'lgID': None,
  'playerID': 'deaneha01',
  'plyrMgr': 'Y',
  'rank': 8,
  'teamID': 'FW1',
  'yearID': 1871}]

## Part 2 - Awards for Managers

Now add a table for the `AwardsManagers.csv` table.

In [19]:
awards = pd.read_csv("./databases/baseball/core/AwardsManagers.csv")
awards.head()

,playerID,awardID,yearID,lgID,tie,notes
0,larusto01,BBWAA Manager of the Year,1983,AL,NaN,NaN
1,lasorto01,BBWAA Manager of the Year,1983,NL,NaN,NaN
2,andersp01,BBWAA Manager of the Year,1984,AL,NaN,NaN
3,freyji99,BBWAA Manager of the Year,1984,NL,NaN,NaN
4,coxbo01,BBWAA Manager of the Year,1985,AL,NaN,NaN


In [20]:
awards.columns

Index(['playerID', 'awardID', 'yearID', 'lgID', 'tie', 'notes'], dtype='object')

In [21]:
[(col, awards[col].is_unique) for col in awards]

[('playerID', False),
 ('awardID', False),
 ('yearID', False),
 ('lgID', False),
 ('tie', False),
 ('notes', False)]

In [22]:
from dfply import mutate
awards = (awards >>
            mutate(id = awards.index))

In [23]:
managers.id.is_unique

True

In [24]:
awards.dtypes

playerID    object
awardID     object
yearID       int64
lgID        object
tie         object
notes       object
id           int64
dtype: object

In [25]:
awards.shape

(179, 7)

In [26]:
awards.head()

,playerID,awardID,yearID,lgID,tie,notes,id
0,larusto01,BBWAA Manager of the Year,1983,AL,NaN,NaN,0
1,lasorto01,BBWAA Manager of the Year,1983,NL,NaN,NaN,1
2,andersp01,BBWAA Manager of the Year,1984,AL,NaN,NaN,2
3,freyji99,BBWAA Manager of the Year,1984,NL,NaN,NaN,3
4,coxbo01,BBWAA Manager of the Year,1985,AL,NaN,NaN,4


In [27]:
#Creat a sqlalchemy types dict
from sqlalchemy import String, Integer
sql_types = {'id':Integer,
             'playerID':String,
             'awardID':String,
             'yearID':Integer,
             'lgID':String,
             'tie':String,
             'notes':String}

# <font color='red'>Warning: Delete database before continuing</font>

In [28]:
#Create schema
#Note: will use the same engine as before
schema = pd.io.sql.get_schema(awards, 'award', keys='id', con=mang_eng, dtype=sql_types)
print(schema)


CREATE TABLE award (
	"playerID" VARCHAR, 
	"awardID" VARCHAR, 
	"yearID" INTEGER, 
	"lgID" VARCHAR, 
	tie VARCHAR, 
	notes VARCHAR, 
	id INTEGER NOT NULL, 
	CONSTRAINT award_pk PRIMARY KEY (id)
)




In [29]:
#Execute the schema
mang_eng.execute(schema)

2019-01-28 08:42:02,648 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE award (
	"playerID" VARCHAR, 
	"awardID" VARCHAR, 
	"yearID" INTEGER, 
	"lgID" VARCHAR, 
	tie VARCHAR, 
	notes VARCHAR, 
	id INTEGER NOT NULL, 
	CONSTRAINT award_pk PRIMARY KEY (id)
)


2019-01-28 08:42:02,650 INFO sqlalchemy.engine.base.Engine ()
2019-01-28 08:42:02,672 INFO sqlalchemy.engine.base.Engine COMMIT


In [30]:
awards.to_sql('award', 
                con=mang_eng, 
                dtype=sql_types, 
                index=False,
                if_exists='append')

2019-01-28 08:44:53,326 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("award")
2019-01-28 08:44:53,328 INFO sqlalchemy.engine.base.Engine ()
2019-01-28 08:44:53,333 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2019-01-28 08:44:53,339 INFO sqlalchemy.engine.base.Engine INSERT INTO award ("playerID", "awardID", "yearID", "lgID", tie, notes, id) VALUES (?, ?, ?, ?, ?, ?, ?)
2019-01-28 08:44:53,341 INFO sqlalchemy.engine.base.Engine (('larusto01', 'BBWAA Manager of the Year', 1983, 'AL', None, None, 0), ('lasorto01', 'BBWAA Manager of the Year', 1983, 'NL', None, None, 1), ('andersp01', 'BBWAA Manager of the Year', 1984, 'AL', None, None, 2), ('freyji99', 'BBWAA Manager of the Year', 1984, 'NL', None, None, 3), ('coxbo01', 'BBWAA Manager of the Year', 1985, 'AL', None, None, 4), ('herzowh01', 'BBWAA Manager of the Year', 1985, 'NL', None, None, 5), ('mcnamjo99', 'BBWAA Manager of the Year', 1986, 'AL', None, None, 6), ('lanieha01', 'BBWAA Manager of the Year', 1986, 'NL', Non

In [31]:
#Query the table!
Session = sessionmaker(mang_eng)
session = Session()

In [33]:
Base = automap_base()
Base.prepare(mang_eng2, reflect=True)
Award = Base.classes.award

In [34]:
stmt = select('*').select_from(Award)
session.execute(stmt).fetchmany(5) >> result_dicts

2019-01-28 08:47:43,296 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2019-01-28 08:47:43,298 INFO sqlalchemy.engine.base.Engine SELECT * 
FROM award
2019-01-28 08:47:43,301 INFO sqlalchemy.engine.base.Engine ()


[{'awardID': 'BBWAA Manager of the Year',
  'id': 0,
  'lgID': 'AL',
  'notes': None,
  'playerID': 'larusto01',
  'tie': None,
  'yearID': 1983},
 {'awardID': 'BBWAA Manager of the Year',
  'id': 1,
  'lgID': 'NL',
  'notes': None,
  'playerID': 'lasorto01',
  'tie': None,
  'yearID': 1983},
 {'awardID': 'BBWAA Manager of the Year',
  'id': 2,
  'lgID': 'AL',
  'notes': None,
  'playerID': 'andersp01',
  'tie': None,
  'yearID': 1984},
 {'awardID': 'BBWAA Manager of the Year',
  'id': 3,
  'lgID': 'NL',
  'notes': None,
  'playerID': 'freyji99',
  'tie': None,
  'yearID': 1984},
 {'awardID': 'BBWAA Manager of the Year',
  'id': 4,
  'lgID': 'AL',
  'notes': None,
  'playerID': 'coxbo01',
  'tie': None,
  'yearID': 1985}]

## Part 3 - Super Hero Powers

Now make a database and table for the super hero powers.

## Problem 1
    
**Task:** One the `super_hero_powers.csv` and verify that the contents of the columns are all Boolean.  In this problem, you need to

1. Create a `dict` that defines the `pandas` column type
2. Read the file in using a `pd.read_csv`.
3. Clean up all the column labels.
    
**Be sure to write clean code!**


In [2]:
powers = pd.read_csv("./data/super_hero_powers.csv")
powers.columns = powers.columns.str.replace(' - ','_')
powers.columns = powers.columns.str.replace('-','_')
powers.columns = powers.columns.str.replace(' ','_')
powers.columns = powers.columns.str.lower()

In [3]:
powers.dtypes[:5]

hero_names               object
agility                    bool
accelerated_healing        bool
lantern_power_ring         bool
dimensional_awareness      bool
dtype: object

In [4]:
sql_types = {}

for col in powers.columns:
    sql_types[col] = Boolean 
sql_types['hero_names'] = String
sql_types

{'accelerated_healing': sqlalchemy.sql.sqltypes.Boolean,
 'adaptation': sqlalchemy.sql.sqltypes.Boolean,
 'agility': sqlalchemy.sql.sqltypes.Boolean,
 'animal_attributes': sqlalchemy.sql.sqltypes.Boolean,
 'animal_control': sqlalchemy.sql.sqltypes.Boolean,
 'animal_oriented_powers': sqlalchemy.sql.sqltypes.Boolean,
 'animation': sqlalchemy.sql.sqltypes.Boolean,
 'anti_gravity': sqlalchemy.sql.sqltypes.Boolean,
 'astral_projection': sqlalchemy.sql.sqltypes.Boolean,
 'astral_travel': sqlalchemy.sql.sqltypes.Boolean,
 'audio_control': sqlalchemy.sql.sqltypes.Boolean,
 'banish': sqlalchemy.sql.sqltypes.Boolean,
 'biokinesis': sqlalchemy.sql.sqltypes.Boolean,
 'camouflage': sqlalchemy.sql.sqltypes.Boolean,
 'changing_armor': sqlalchemy.sql.sqltypes.Boolean,
 'clairvoyance': sqlalchemy.sql.sqltypes.Boolean,
 'cloaking': sqlalchemy.sql.sqltypes.Boolean,
 'cold_resistance': sqlalchemy.sql.sqltypes.Boolean,
 'cryokinesis': sqlalchemy.sql.sqltypes.Boolean,
 'danger_sense': sqlalchemy.sql.sqltype

In [5]:
[(col, powers[col].is_unique) for col in powers][:5]

[('hero_names', True),
 ('agility', False),
 ('accelerated_healing', False),
 ('lantern_power_ring', False),
 ('dimensional_awareness', False)]

## Problem 2
    
Now define an `sqlalchemy` table for these data using `pandas` `to_sql` dataframe method.  You can use the `sqlalchemy.String` and `sqlalchemy.Boolean` columns type, which are [documented here](https://docs.sqlalchemy.org/en/latest/core/type_basics.html)

# <font color='red'>Warning: Delete database before continuing</font>

## Problem 3
    
Now you need to make a new `engine`, `inspect` your database, and make a `session` to query the database.

In [6]:
super_eng = create_engine("sqlite:///databases/super_powers.db")
super_eng.echo = True
schema = pd.io.sql.get_schema(powers, 'power', keys='hero_names', con=super_eng, dtype=sql_types)
print(schema)


CREATE TABLE power (
	hero_names VARCHAR NOT NULL, 
	agility BOOLEAN, 
	accelerated_healing BOOLEAN, 
	lantern_power_ring BOOLEAN, 
	dimensional_awareness BOOLEAN, 
	cold_resistance BOOLEAN, 
	durability BOOLEAN, 
	stealth BOOLEAN, 
	energy_absorption BOOLEAN, 
	flight BOOLEAN, 
	danger_sense BOOLEAN, 
	underwater_breathing BOOLEAN, 
	marksmanship BOOLEAN, 
	weapons_master BOOLEAN, 
	power_augmentation BOOLEAN, 
	animal_attributes BOOLEAN, 
	longevity BOOLEAN, 
	intelligence BOOLEAN, 
	super_strength BOOLEAN, 
	cryokinesis BOOLEAN, 
	telepathy BOOLEAN, 
	energy_armor BOOLEAN, 
	energy_blasts BOOLEAN, 
	duplication BOOLEAN, 
	size_changing BOOLEAN, 
	density_control BOOLEAN, 
	stamina BOOLEAN, 
	astral_travel BOOLEAN, 
	audio_control BOOLEAN, 
	dexterity BOOLEAN, 
	omnitrix BOOLEAN, 
	super_speed BOOLEAN, 
	possession BOOLEAN, 
	animal_oriented_powers BOOLEAN, 
	weapon_based_powers BOOLEAN, 
	electrokinesis BOOLEAN, 
	darkforce_manipulation BOOLEAN, 
	death_touch BOOLEAN, 
	teleportati

In [7]:
super_eng.execute(schema)

2019-01-28 19:35:21,456 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2019-01-28 19:35:21,458 INFO sqlalchemy.engine.base.Engine ()
2019-01-28 19:35:21,462 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2019-01-28 19:35:21,464 INFO sqlalchemy.engine.base.Engine ()
2019-01-28 19:35:21,467 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE power (
	hero_names VARCHAR NOT NULL, 
	agility BOOLEAN, 
	accelerated_healing BOOLEAN, 
	lantern_power_ring BOOLEAN, 
	dimensional_awareness BOOLEAN, 
	cold_resistance BOOLEAN, 
	durability BOOLEAN, 
	stealth BOOLEAN, 
	energy_absorption BOOLEAN, 
	flight BOOLEAN, 
	danger_sense BOOLEAN, 
	underwater_breathing BOOLEAN, 
	marksmanship BOOLEAN, 
	weapons_master BOOLEAN, 
	power_augmentation BOOLEAN, 
	animal_attributes BOOLEAN, 
	longevity BOOLEAN, 
	intelligence BOOLEAN, 
	super_strength BOOLEAN, 
	cryokinesis BOOLEAN, 
	telepathy BOOLEAN, 
	energy_armor BOOL

2019-01-28 19:35:21,469 INFO sqlalchemy.engine.base.Engine ()
2019-01-28 19:35:21,494 INFO sqlalchemy.engine.base.Engine COMMIT


In [8]:
powers.to_sql('power',
             con=super_eng,
             dtype=sql_types,
             index=False,
             if_exists='append')

2019-01-28 19:35:22,688 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("power")
2019-01-28 19:35:22,689 INFO sqlalchemy.engine.base.Engine ()
2019-01-28 19:35:22,701 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2019-01-28 19:35:22,871 INFO sqlalchemy.engine.base.Engine INSERT INTO power (hero_names, agility, accelerated_healing, lantern_power_ring, dimensional_awareness, cold_resistance, durability, stealth, energy_absorption, flight, danger_sense, underwater_breathing, marksmanship, weapons_master, power_augmentation, animal_attributes, longevity, intelligence, super_strength, cryokinesis, telepathy, energy_armor, energy_blasts, duplication, size_changing, density_control, stamina, astral_travel, audio_control, dexterity, omnitrix, super_speed, possession, animal_oriented_powers, weapon_based_powers, electrokinesis, darkforce_manipulation, death_touch, teleportation, enhanced_senses, telekinesis, energy_beams, magic, hyperkinesis, jump, clairvoyance, dimensional_travel, p

2019-01-28 19:35:22,898 INFO sqlalchemy.engine.base.Engine COMMIT


In [9]:
Session = sessionmaker(super_eng)
session = Session()

In [10]:
Base = automap_base()
Base.prepare(super_eng, reflect = True)
Power = Base.classes.power

2019-01-28 19:37:23,512 INFO sqlalchemy.engine.base.Engine SELECT name FROM sqlite_master WHERE type='table' ORDER BY name
2019-01-28 19:37:23,514 INFO sqlalchemy.engine.base.Engine ()
2019-01-28 19:37:23,519 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("power")
2019-01-28 19:37:23,520 INFO sqlalchemy.engine.base.Engine ()
2019-01-28 19:37:23,591 INFO sqlalchemy.engine.base.Engine SELECT sql FROM  (SELECT * FROM sqlite_master UNION ALL   SELECT * FROM sqlite_temp_master) WHERE name = 'power' AND type = 'table'
2019-01-28 19:37:23,593 INFO sqlalchemy.engine.base.Engine ()
2019-01-28 19:37:23,598 INFO sqlalchemy.engine.base.Engine PRAGMA foreign_key_list("power")
2019-01-28 19:37:23,599 INFO sqlalchemy.engine.base.Engine ()
2019-01-28 19:37:23,601 INFO sqlalchemy.engine.base.Engine SELECT sql FROM  (SELECT * FROM sqlite_master UNION ALL   SELECT * FROM sqlite_temp_master) WHERE name = 'power' AND type = 'table'
2019-01-28 19:37:23,605 INFO sqlalchemy.engine.base.Engine ()
2019-01

In [11]:
stmt = select('*').select_from(Power)
session.execute(stmt).fetchmany(5) >> result_dicts

2019-01-28 19:38:30,504 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2019-01-28 19:38:30,506 INFO sqlalchemy.engine.base.Engine SELECT * 
FROM power
2019-01-28 19:38:30,508 INFO sqlalchemy.engine.base.Engine ()


[{'accelerated_healing': 0,
  'adaptation': 0,
  'agility': 1,
  'animal_attributes': 0,
  'animal_control': 0,
  'animal_oriented_powers': 0,
  'animation': 0,
  'anti_gravity': 0,
  'astral_projection': 0,
  'astral_travel': 0,
  'audio_control': 0,
  'banish': 0,
  'biokinesis': 0,
  'camouflage': 0,
  'changing_armor': 0,
  'clairvoyance': 0,
  'cloaking': 0,
  'cold_resistance': 0,
  'cryokinesis': 0,
  'danger_sense': 0,
  'darkforce_manipulation': 0,
  'death_touch': 0,
  'density_control': 0,
  'dexterity': 0,
  'dimensional_awareness': 0,
  'dimensional_travel': 0,
  'duplication': 0,
  'durability': 0,
  'echolocation': 0,
  'elasticity': 0,
  'electrical_transport': 0,
  'electrokinesis': 0,
  'element_control': 0,
  'elemental_transmogrification': 0,
  'empathy': 0,
  'energy_absorption': 0,
  'energy_armor': 0,
  'energy_beams': 0,
  'energy_blasts': 0,
  'energy_constructs': 0,
  'energy_manipulation': 0,
  'energy_resistance': 0,
  'enhanced_hearing': 0,
  'enhanced_memo

## Problem 4
    
Perform `sqlalchemy` queries to answer each of the following questions.

1. How many heroes have both Super Strength and Super Speed?
2. How many heroes have names that start with the word *Black*
3. Are heroes with Agility more likely to have Stealth?
4. What fraction of all heroes that can fly also have Super Strength?
5. Consider heroes that have names that contain `"girl"`, `"boy"`, `"woman"`, or `"man"`.  Compute the following ratio

$$\frac{N(\text{boy or man})}{N(\text{girl or woman}}$$

**Hint:** You will need to use some combination of `where`, `group_by`, and `count` for each part.

In [27]:
stmt1 = select([func.count(Power.hero_names).label('hero_count')]).\
        where(Power.super_strength == 1).\
        where(Power.super_speed == 1)
session.execute(stmt1).fetchall() >> result_dicts

2019-01-28 19:53:15,907 INFO sqlalchemy.engine.base.Engine SELECT count(power.hero_names) AS hero_count 
FROM power 
WHERE power.super_strength = ? AND power.super_speed = ?
2019-01-28 19:53:15,910 INFO sqlalchemy.engine.base.Engine (1, 1)


[{'hero_count': 219}]

### 219 Heroes have both Super Strength and Super Speed

In [28]:
stmt2 = select([func.count(Power.hero_names).label('hero_count')]).\
        where(Power.hero_names.startswith('Black'))
session.execute(stmt2).fetchall() >> result_dicts

2019-01-28 19:58:21,584 INFO sqlalchemy.engine.base.Engine SELECT count(power.hero_names) AS hero_count 
FROM power 
WHERE (power.hero_names LIKE ? || '%%')
2019-01-28 19:58:21,587 INFO sqlalchemy.engine.base.Engine ('Black',)


[{'hero_count': 16}]

### 16 Heroes have names starting with 'Black'

## Problem 5

Tell me another cool fact about the super powers.